In [ ]:
import requests
import time
import json
from datetime import datetime, timedelta
    
def parse_rbc_articles(url):
    headers = {
        "accept": "application/json, text/javascript, */*; q=0.01",
        "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
        "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "x-requested-with": "XMLHttpRequest"
    }

    status_code = -1
    response = None
    while status_code == -1:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                return json.dumps({"error": "Failed to fetch the data from the URL provided."})
            status_code = response.status_code
        except Exception as e:
            time.sleep(30)

    data = response.json()
    articles_list = []
    for item in data['items']:
        article = {
            "header": item.get('title', ''),
            "published": datetime.fromtimestamp(item.get('publish_date_t', 0)).strftime(
                '%Y-%m-%d %H:%M:%S') if item.get('publish_date_t') else '',
            "summary": item.get('body', ''),
            "ticker": ''
        }
        articles_list.append(article)

    return articles_list

def parse_tradingview_headlines(url):
    headers = {
        "accept": "*/*",
        "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
        "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "\"Windows\"",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-site"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return json.dumps({"error": "Failed to fetch the data from the URL provided."})

    data = response.json()
    headlines_list = []
    for item in data['items']:
        publish_time = datetime.fromtimestamp(item['published']).strftime('%Y-%m-%d %H:%M:%S') if item.get(
            'published') else ''
        ticker = ', '.join([symbol['symbol'] for symbol in item.get('relatedSymbols', []) if 'symbol' in symbol])

        headline = {
            "header": item.get('title', ''),
            "published": publish_time,
            "summary": '',
            "ticker": ticker
        }
        headlines_list.append(headline)

    return headlines_list

def parse_url(url):
    if "rbc" in url:
        return parse_rbc_articles(url)
    elif "tradingview" in url:
        return parse_tradingview_headlines(url)
    else:
        return []


In [ ]:
TOKEN = "tinkof api token"

In [ ]:
from tinkoff.invest import RequestError, Quotation, Client, CandleInterval
from tinkoff.invest.grpc.orders_pb2 import ORDER_DIRECTION_BUY, ORDER_DIRECTION_SELL, ORDER_TYPE_BESTPRICE
from tinkoff.invest.utils import now


def convert_quotation_to_float(units: int, nano: int):
    return units + (nano / (10 ** 9))


def float_to_quotation(f: float) -> Quotation:
    float_str = str(f)
    units = int(float_str[:float_str.find('.')])

    if len(float_str[float_str.find('.')+1:]) > 9:
        end = 9 - len(float_str[float_str.find('.')+1:])
        nano = int(float_str[float_str.find('.')+1:end])
    elif len(str(float_str[float_str.find('.')+1:])) < 9:
        nano = int(int(int(float_str[float_str.find('.')+1:]) * 1_000_000_000) // pow(10, 9 - (9 - len(float_str[float_str.find('.')+1:]))))
    else:
        nano = int(float_str[float_str.find('.')+1:])

    print("Str: " + float_str + " units: " + str(units) + " nano: " + str(nano))
    return Quotation(units=units, nano=nano)


def round_to_near_min_increment(min_inc, price):
    return price - (price % min_inc)


In [ ]:
name_to_ticker = {}
ticker_to_uid = {}
uid_to_ticker = {}
lots = {}
min_price_increment = {}

with Client(TOKEN) as client:
    for method in ['shares']:
        for item in getattr(client.instruments, method)().instruments:
            if item.for_qual_investor_flag is False and item.api_trade_available_flag is True and item.buy_available_flag is True and item.currency == 'rub' and item.country_of_risk == 'RU':
                name_to_ticker[item.name] = item.ticker
                ticker_to_uid[item.ticker] = item.uid
                uid_to_ticker[item.uid] = item.ticker
                lots[item.ticker] = item.lot
                min_price_increment[item.ticker] = convert_quotation_to_float(
                    units=item.min_price_increment.units, nano=item.min_price_increment.nano)

In [ ]:
# 200 новостей от trading view
tv_articles = {}
for ticker in ticker_to_uid.keys():
    print(ticker)
    news = parse_url("https://news-headlines.tradingview.com/v2/view/headlines/symbol?client=web&lang=ru&streaming=true&symbol=MOEX%3A" + ticker)
    tv_articles[ticker] = []
    for item in news:
        try:
            published_dt = datetime.strptime(item['published'], '%Y-%m-%d %H:%M:%S')
            tv_articles[ticker].append((item['header'], published_dt))
            print(f"New article found: {item['header']} published at {item['published']}")
        except Exception as e:
            print(e)
            continue
    time.sleep(5)

In [ ]:
import os
import csv
from datetime import datetime


# Проверяем существует ли папка 'data', и если нет, создаем ее
if not os.path.exists('data'):
    os.makedirs('data')

for ticker, articles in tv_articles.items():
    # Изменяем путь к файлу для сохранения в папке 'data'
    filename = os.path.join('data', f"tv_{ticker}.csv")
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['header', 'published'])  # Запись заголовков столбцов
        for header, published_dt in articles:
            # Преобразуем datetime объект обратно в строку для записи
            published_str = published_dt.strftime('%Y-%m-%d %H:%M:%S')
            writer.writerow([header, published_str])


In [ ]:
# проверям недоступные имена. Если недоступно, то печатаем и удаляем из списка.

direct_available_names = {}
headers = {
    "accept": "*/*",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site"
}

for company_name, ticker in name_to_ticker.items():
    url = "https://www.rbc.ru/tags/?tag=" + company_name
    #url = "https://www.rbc.ru/search/ajax/?tag=" + company_name + "&project=rbcnews&page=1"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(company_name + " not avaliable  ticker: " + ticker)
    else:
        direct_available_names[company_name] = ticker
        print(company_name + " available")

print(direct_available_names)
# руками вставляем нужные акции

In [ ]:
direct_available_names["En+ Group"] = "ENPG"
direct_available_names["ВКонтакте"] = "VKCO"
direct_available_names["«Таттелеком»"] = "TTLK"
direct_available_names["ММК"] = "MAGN"
direct_available_names["ВТБ"] = "VTBR"
direct_available_names["ЦИАН"] = "CIAN"
direct_available_names["Сбербанк"] = "SBER"
direct_available_names["X5"] = "FIVE"
direct_available_names["Черкизово"] = "GCHE"
direct_available_names["Тинькофф Банк"] = "TCSG"
direct_available_names["Ozon"] = "OZON"

In [ ]:
rbc_news_artictles = {}

for company_name, ticker in direct_available_names.items():
    rbc_news_artictles[ticker] = []
    page = 0
    news = parse_url("https://www.rbc.ru/search/ajax/?tag=" + company_name + "&project=rbcnews&page=" + str(page))
    while len(news) != 0:
        for item in news:
            try:
                published_dt = datetime.strptime(item['published'], '%Y-%m-%d %H:%M:%S')
                rbc_news_artictles[ticker].append((item['header'], published_dt))
                print(f"New article found: {item['header']} published at {item['published']}")
            except Exception as e:
                print(e)
                time.sleep(10)
                continue
        page += 1
        news = parse_url("https://www.rbc.ru/search/ajax/?tag=" + company_name + "&project=rbcnews&page=" + str(page))
        time.sleep(1)
    time.sleep(5)

In [ ]:
import os
import csv
from datetime import datetime


# Проверяем существует ли папка 'data', и если нет, создаем ее
if not os.path.exists('data'):
    os.makedirs('data')

for ticker, articles in rbc_news_artictles.items():
    # Изменяем путь к файлу для сохранения в папке 'data'
    filename = os.path.join('data', f"rbc_news_{ticker}.csv")
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['header', 'published'])  # Запись заголовков столбцов
        for header, published_dt in articles:
            # Преобразуем datetime объект обратно в строку для записи
            published_str = published_dt.strftime('%Y-%m-%d %H:%M:%S')
            writer.writerow([header, published_str])


In [ ]:
#!pip install pymorphy2

In [ ]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

def generate_declensions_with_full_name(name, stop_words):
    declensions = set([name.lower()])  # Добавляем полное название в нижнем регистре
    for word in name.split():
        word = word.lower()  # Приведение к нижнему регистру
        if word not in stop_words and len(word) > 3:  # Игнорирование стоп-слов
            parsed_word = morph.parse(word)[0]
            declensions.add(parsed_word.normal_form)
            # Добавление всех форм слова в набор склонений
            for form in parsed_word.lexeme:
                declensions.add(form.word)
    return declensions


In [ ]:
stop_words = {"акции", "привилегированные", "обыкновенные", "и", "-", "банк", "гдр", "group", "группа", "мир", "мира", "биржа", 'компания',
             "красный", "октябрь", "сеть", "сети", "рынка", "сетями", "московская", "московской", "москва", "московский", "адр", "второй",
              "вторая", "интерфакс", "система", "в", "московском", "регион", "нефть"}

# Генерация склонений для каждой компании и сохранение их в новый словарь
company_declensions = {company: generate_declensions_with_full_name(company, stop_words) for company in direct_available_names.keys()}
for company in name_to_ticker.keys():
    additional_declensions = generate_declensions_with_full_name(company, stop_words)
    # Объединение наборов склонений, если компания уже присутствует в словаре
    if company in company_declensions:
        company_declensions[company] = company_declensions[company].union(additional_declensions)
    else:
        company_declensions[company] = additional_declensions

In [ ]:
import string

def preprocess_text(text):
    # Добавление кавычек в список удаляемых символов
    punctuation_extended = string.punctuation + '«»“”‘’'  # Учитывает разные типы кавычек
    return text.translate(str.maketrans('', '', punctuation_extended)).lower()

def check_companies_in_text(text, stop_words):
    text = preprocess_text(" " + text.lower() + " ")  # Предобработка и приведение к нижнему регистру
    found_companies = {}

    for company, declensions in company_declensions.items():
        # Проверка на полное вхождение названия компании в текст
        if " " + company.lower() + " " in text:
            # Если полное название компании найдено, добавляем его в результат
            if company in direct_available_names:
                found_companies[company] = direct_available_names[company]
            elif company in name_to_ticker:
                found_companies[company] = name_to_ticker[company]
        else:
            # Проверка на вхождение склоненных форм названия в тексте
            for declension in declensions:
                if " " + declension.lower() + " " in text:  # Изменено на проверку вхождения во всём тексте
                    if company in direct_available_names:
                        found_companies[company] = direct_available_names[company]
                    elif company in name_to_ticker:
                        found_companies[company] = name_to_ticker[company]
                    break  # Прерываем цикл после первого найденного совпадения

    return found_companies


text = "Сегодня акции Fix Price показали рост, также как и бумаги Селигдар."
found_companies = check_companies_in_text(text, stop_words)
print("Найденные компании и тикеры:", found_companies)


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# Заголовки запроса
headers = {
    "accept": "*/*",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "sec-ch-ua": "\"Not A(Brand\";v=\"99\", \"Google Chrome\";v=\"121\", \"Chromium\";v=\"121\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site"
}

alenka_news_artictles = {}

# Начальная дата
date = datetime(2024, 3, 7)

# Конечная дата
end_date = datetime(2020, 1, 1)

# Пока текущая дата больше или равна конечной дате
while date >= end_date:
    try:
        # Формируем URL с текущей датой
        formatted_date = date.strftime('%d.%m.%Y')
        url = f"https://alenka.capital/category/view_596/?date={formatted_date}"

        # Отправляем HTTP запрос
        response = requests.get(url, headers=headers)  # Предполагается, что headers уже определены

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            news_items = soup.find_all('li', class_='feed__item')

            for item in news_items:
                time_published = item.find('time', class_='feed__date').text.strip()
                news_title = item.find('h2', class_='feed__text').text.strip()

                # Предполагается, что функция check_companies_in_text и словарь alenka_news_artictles уже определены
                related_tickers = check_companies_in_text(news_title, stop_words)
                published_dt = datetime.strptime(time_published, '%d.%m.%Y, %H:%M')
                formatted_published_dt = published_dt.strftime('%Y-%m-%d %H:%M:%S')

                for ticker in related_tickers.values():
                    if ticker not in alenka_news_artictles:
                        alenka_news_artictles[ticker] = []
                    alenka_news_artictles[ticker].append((news_title, formatted_published_dt))
                print(f"{time_published} - {news_title}")
        else:
            print(f"Страница за {formatted_date} не доступна")

        # Уменьшаем дату на один день
        date -= timedelta(days=1)
        time.sleep(1)
    except Exception as e:
        print(e)
        time.sleep(30)

In [ ]:
import os
import csv
from datetime import datetime


# Проверяем существует ли папка 'data', и если нет, создаем ее
if not os.path.exists('data'):
    os.makedirs('data')

for ticker, articles in alenka_news_artictles.items():
    # Изменяем путь к файлу для сохранения в папке 'data'
    filename = os.path.join('data', f"alenka_news_{ticker}.csv")
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['header', 'published'])  # Запись заголовков столбцов
        for header, published_dt in articles:
            writer.writerow([header, published_dt])


In [2]:
from datetime import datetime, timedelta
import requests
import time

tass_news_articles = {}

headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
    "sec-ch-ua": "\"Chromium\";v=\"122\", \"Not(A:Brand\";v=\"24\", \"Google Chrome\";v=\"122\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin"
}

last_updated = "2024-03-10T00:00:00.028547"
end_date = datetime(2022, 1, 1)
last_valid_datetime = None
max_backward_jump = timedelta(days=2)  # Максимально допустимый скачок назад (например, 1 год)

while True:
    try:
        url = f"https://tass.ru/tbp/api/v1/content?limit=20&lang=ru&rubrics=/ekonomika&sort=-es_updated_dt&last_es_updated_dt={last_updated}"
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            data = response.json()
            
            if not data['result']:
                break  # Выходим из цикла, если новостей больше нет
            
            first_news_dt = datetime.fromisoformat(data['result'][0]['published_dt'].rstrip("Z"))
            if last_valid_datetime and first_news_dt > last_valid_datetime:
                # Если время "соскочило" вперед, корректируем на час назад и повторяем запрос
                last_updated = (last_valid_datetime - timedelta(hours=1)).isoformat() + "Z"
                continue
                
            if last_valid_datetime and first_news_dt < last_valid_datetime - max_backward_jump:
                # Корректируем last_updated на один час вперед, чтобы попытаться избежать большого скачка назад
                last_updated = (last_valid_datetime - timedelta(hours=1)).isoformat() + "Z"
                continue
                
            for item in data['result']:
                title = item['title']
                published_dt = item['published_dt']
                published_dt_obj = datetime.fromisoformat(published_dt.rstrip("Z"))
                formatted_published_dt = published_dt_obj.strftime('%Y-%m-%d %H:%M:%S')
                
                related_tickers = check_companies_in_text(title, stop_words)

                for ticker in related_tickers.values():
                    if ticker not in tass_news_articles:
                        tass_news_articles[ticker] = []
                    tass_news_articles[ticker].append((title, formatted_published_dt))

                
                print(f"Заголовок: {title}\nДата публикации: {formatted_published_dt}\n")
                
            # Обновляем last_updated для следующего запроса
            last_updated = data['result'][-1]['es_updated_dt']
            last_valid_datetime = datetime.fromisoformat(last_updated.rstrip("Z"))
            
            if last_valid_datetime <= end_date:
                break  # Если достигли или перешли конечную дату, завершаем цикл
        
        #time.sleep(0.2)  # Задержка для соблюдения лимитов API
    except Exception as e:
        print(e)
        time.sleep(30)  # Задержка при возникновении исключения

name 'check_companies_in_text' is not defined


KeyboardInterrupt: 

In [ ]:
import os
import csv
from datetime import datetime


# Проверяем существует ли папка 'data', и если нет, создаем ее
if not os.path.exists('data'):
    os.makedirs('data')

for ticker, articles in tass_news_articles.items():
    # Изменяем путь к файлу для сохранения в папке 'data'
    filename = os.path.join('data', f"tass_news_{ticker}.csv")
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['header', 'published'])  # Запись заголовков столбцов
        for header, published_dt in articles:
            writer.writerow([header, published_dt])


In [ ]:
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup

# Функция для получения новостей с сайта Interfax за конкретный день
def get_interfax_news_for_day(date):
    date_str = date.strftime("%Y/%m/%d")
    url = f"https://www.interfax.ru/business/news/{date_str}"
    response = requests.get(url)
    news_list = []

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        news_items = soup.find_all('div', attrs={'data-id': True})

        for item in news_items:
            time_span = item.find('span')
            # Форматируем время с добавлением 0 секунд для единобразия
            news_time = datetime.strptime(f"{date_str} {time_span.text}:00", "%Y/%m/%d %H:%M:%S")
            title_h3 = item.find('h3')
            if title_h3:
                news_title = title_h3.text
                # Используем strftime для форматирования datetime объекта в строку с добавлением 0 секунд
                formatted_news_time = news_time.strftime('%Y-%m-%d %H:%M:%S')
                news_list.append({'title': news_title, 'datetime': formatted_news_time})

    return news_list

# Функция для получения новостей в заданном диапазоне дат
def get_interfax_news(start_date, end_date):
    delta = timedelta(days=1)
    current_date = start_date
    all_news = []

    while current_date <= end_date:
        daily_news = get_interfax_news_for_day(current_date)
        all_news.extend(daily_news)
        current_date += delta

    return all_news

# Пример использования функций
# Обратите внимание, что вызовы функций закомментированы, чтобы предотвратить их выполнение здесь.
start_date = datetime(2022, 1, 1)
end_date = datetime(2024, 3, 9)
news = get_interfax_news(start_date, end_date)

In [ ]:
interfacs_news_articles = {}
for current_news in news:
    print("Заголовок: " + current_news["title"] + "\nДата публикации: " + current_news["datetime"] + "\n")
    related_tickers = check_companies_in_text(current_news["title"], stop_words)
    for ticker in related_tickers.values():
        if ticker not in interfacs_news_articles:
            interfacs_news_articles[ticker] = []
        interfacs_news_articles[ticker].append((current_news["title"], current_news["datetime"]))

In [ ]:
#Поможет распарсить 200 новостей на каждую акцию с trading view используя тикеры от тинькофф.
#news = parse_url("https://news-headlines.tradingview.com/v2/view/headlines/symbol?client=web&lang=ru&streaming=true&symbol=MOEX%3ASBER")

#Вводим список компаний руками, нужно автоматическое преобразование к hex, потом просто перебираем страницы, до конца.
#news = parse_url("https://www.rbc.ru/search/ajax/?tag=%D0%A1%D0%B1%D0%B5%D1%80%D0%B1%D0%B0%D0%BD%D0%BA&project=rbcnews&page=70")

#№https://alenka.capital/category/view_596/?date=01.02.2024

for item in news:
    try:
        published_dt = datetime.strptime(item['published'], '%Y-%m-%d %H:%M:%S')
        print(f"New article found: {item['header']} published at {item['published']}")
    except Exception as e:
        print(e)
        continue
print(len(news))

In [ ]:
# Фильтровать новости: аноснс событий, дайджест, лидеры роста и падаения